In [2]:
import pandas as pd

# Load the uploaded Excel file
# file_path = '/mnt/data/RDA.xlsx'
file_path = r"C:\Users\ireti\Downloads\RDA.xlsx"
excel_data = pd.ExcelFile(file_path)

# List all sheet names to understand the structure
sheet_names = excel_data.sheet_names
sheet_names


['Popout  Stroop Test PE',
 'Visual Search PE',
 'Math MCQ PE',
 'Math ToF PE',
 'Verbal MCQ PE',
 'SRT PE',
 'CRT PE']

In [16]:
import pandas as pd
from scipy.stats import shapiro

# Function to load data from each sheet and perform the Shapiro-Wilk test for normality
def check_normality(sheet_name):
    data = excel_data.parse(sheet_name)
    p_values = []
    p_values = {}
    
    # Assuming we want to check normality for each column (variable)
    for column in data.columns:
        if data[column].dtype in ['float64', 'int64']:  # Test numerical data only
            stat, p = shapiro(data[column].dropna())  # Perform Shapiro-Wilk test
            # p_values.append((column, p))
            p_values[column] = p
    
    return p_values

# Checking normality for each sheet
normality_results = {sheet: check_normality(sheet) for sheet in sheet_names}
pd.DataFrame(normality_results).T

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20
Popout Stroop Test PE,0.001745,8.868342e-13,7.285050e-05,6.266703e-04,2.336482e-01,1.334648e-08,2.231714e-04,6.719707e-12,5.472185e-03,1.933907e-03,1.628302e-10,1.036222e-04,7.841562e-09,5.786906e-11,8.351213e-08,2.339538e-04,7.746309e-02,0.000184,1.097605e-09,0.501845
Visual Search PE,0.002023,6.319270e-08,6.386908e-05,1.190614e-16,1.050017e-05,4.260883e-13,1.535838e-06,4.025674e-11,4.595335e-15,1.589133e-09,2.107163e-04,9.659863e-04,1.796323e-07,2.697010e-12,5.399016e-16,2.129956e-08,1.852340e-10,0.000017,1.338484e-10,0.070497
Math MCQ PE,0.849152,2.079409e-01,1.013824e-04,2.297716e-04,2.512391e-01,2.619633e-02,4.822748e-02,1.338948e-03,2.318221e-02,1.903662e-04,1.962336e-02,9.669272e-03,8.622967e-01,4.750782e-03,3.617772e-04,2.791663e-02,1.351338e-03,0.966690,6.837177e-01,0.159977
Math ToF PE,0.013478,1.355445e-01,3.227909e-02,4.870741e-03,5.162402e-01,5.357891e-03,2.818415e-01,1.969473e-01,5.442235e-01,6.035951e-03,4.830525e-01,5.243239e-01,9.633292e-02,7.029381e-02,5.339610e-03,3.696711e-01,3.268572e-02,0.079046,7.582733e-03,0.004593
Verbal MCQ PE,0.418192,2.298629e-01,4.890533e-02,5.428239e-03,3.704679e-02,8.942034e-01,1.037014e-01,5.097558e-01,2.389645e-01,1.565693e-02,2.176586e-01,2.975696e-04,5.696384e-01,5.093705e-02,7.736452e-03,9.462428e-02,2.407493e-01,0.011283,2.152513e-01,0.047448
SRT PE,0.000350,5.736828e-02,1.161913e-04,1.472193e-02,2.039196e-10,1.875934e-03,3.241382e-10,7.779619e-06,5.823583e-01,4.569856e-04,7.092580e-10,8.270675e-04,1.471854e-11,3.596153e-05,6.611567e-12,1.670942e-06,1.918516e-07,0.000070,1.228588e-02,0.000628
CRT PE,0.013036,7.068516e-02,8.317320e-11,3.524812e-03,1.326382e-06,5.258304e-06,1.276606e-03,4.043136e-04,5.653731e-05,1.007609e-02,1.558993e-10,2.154145e-08,5.803119e-02,6.336447e-07,1.001847e-04,1.584396e-12,4.950637e-03,0.000935,1.309995e-03,0.000001


In [5]:
from scipy.stats import ttest_ind, mannwhitneyu

# Function to perform the appropriate test based on normality
def compute_significance_test(sheet_name, normality_results):
    data = excel_data.parse(sheet_name)
    p_values = {}
    
    for column, normality_p in normality_results:
        if normality_p >= 0.05:  # Normally distributed, use t-test
            stat, p_value = ttest_ind(data[column].dropna(), data[column].dropna())  # Assuming comparison within the column
        else:  # Non-normally distributed, use Mann-Whitney U test
            stat, p_value = mannwhitneyu(data[column].dropna(), data[column].dropna())

        p_values[column] = p_value

    return p_values

# Applying the tests on all datasets
significance_results = {sheet: compute_significance_test(sheet, normality_results[sheet]) for sheet in sheet_names}
significance_results


{'Popout  Stroop Test PE': {'P1': 1.0,
  'P2': 1.0,
  'P3': 1.0,
  'P4': 1.0,
  'P5': 1.0,
  'P6': 1.0,
  'P7': 1.0,
  'P8': 1.0,
  'P9': 1.0,
  'P10': 1.0,
  'P11': 1.0,
  'P12': 1.0,
  'P13': 1.0,
  'P14': 1.0,
  'P15': 1.0,
  'P16': 1.0,
  'P17': 1.0,
  'P18': 1.0,
  'P19': 1.0,
  'P20': 1.0},
 'Visual Search PE': {'P1': 1.0,
  'P2': 1.0,
  'P3': 1.0,
  'P4': 1.0,
  'P5': 1.0,
  'P6': 1.0,
  'P7': 1.0,
  'P8': 1.0,
  'P9': 1.0,
  'P10': 1.0,
  'P11': 1.0,
  'P12': 1.0,
  'P13': 1.0,
  'P14': 1.0,
  'P15': 1.0,
  'P16': 1.0,
  'P17': 1.0,
  'P18': 1.0,
  'P19': 1.0,
  'P20': 1.0},
 'Math MCQ PE': {'P1': 1.0,
  'P2': 1.0,
  'P3': 1.0,
  'P4': 1.0,
  'P5': 1.0,
  'P6': 1.0,
  'P7': 1.0,
  'P8': 1.0,
  'P9': 1.0,
  'P10': 1.0,
  'P11': 1.0,
  'P12': 1.0,
  'P13': 1.0,
  'P14': 1.0,
  'P15': 1.0,
  'P16': 1.0,
  'P17': 1.0,
  'P18': 1.0,
  'P19': 1.0,
  'P20': 1.0},
 'Math ToF PE': {'P1': 1.0,
  'P2': 1.0,
  'P3': 1.0,
  'P4': 1.0,
  'P5': 1.0,
  'P6': 1.0,
  'P7': 1.0,
  'P8': 1.0,
  'P

In [12]:
df = pd.DataFrame(significance_results).T
df

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20
Popout Stroop Test PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Visual Search PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Math MCQ PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Math ToF PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Verbal MCQ PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
SRT PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CRT PE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


1. Check for normality in each column using a Shapiro-Wilk test.
2. If normal: Perform a One-Way ANOVA across the columns.
3. If not normal: Use the Kruskal-Wallis test.

In [7]:
import pandas as pd
from scipy.stats import shapiro, f_oneway, kruskal

# Load the dataset
file_path = r"C:\Users\ireti\Downloads\RDA.xlsx"
xls = pd.ExcelFile(file_path)

# Function to check normality across all columns of a dataframe
def check_normality(df):
    # We perform the Shapiro-Wilk test for normality across all columns
    normality_p_values = []
    for col in df.columns:
        stat, p_value = shapiro(df[col].dropna())  # Shapiro-Wilk test
        normality_p_values.append(p_value)
    return normality_p_values

# Function to apply either ANOVA or Kruskal-Wallis based on normality
def overall_test(df):
    normality_p_values = check_normality(df)
    
    # If all columns have p-value > 0.05, we assume normal distribution
    if all(p > 0.05 for p in normality_p_values):
        # Apply One-Way ANOVA across all columns
        stat, p_value = f_oneway(*[df[col].dropna() for col in df.columns])
        test_used = 'One-Way ANOVA (Normal Distribution)'
    else:
        # Apply Kruskal-Wallis Test (non-parametric)
        stat, p_value = kruskal(*[df[col].dropna() for col in df.columns])
        test_used = 'Kruskal-Wallis Test (Non-Normal Distribution)'
    
    return test_used, p_value

# Iterate over all sheets and perform the tests
test_results = {}
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    test, p_value = overall_test(df)
    test_results[sheet] = {'Test Used': test, 'p-value': p_value}

test_results


{'Popout  Stroop Test PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 1.9429008486621662e-83},
 'Visual Search PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 2.724569313921508e-24},
 'Math MCQ PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 1.6198287594794706e-06},
 'Math ToF PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 6.412447000961338e-10},
 'Verbal MCQ PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 6.94006409058878e-12},
 'SRT PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 5.641681054816329e-105},
 'CRT PE': {'Test Used': 'Kruskal-Wallis Test (Non-Normal Distribution)',
  'p-value': 1.6502670019697278e-73}}

In [17]:
df = pd.DataFrame(test_results)
df

,Popout Stroop Test PE,Visual Search PE,Math MCQ PE,Math ToF PE,Verbal MCQ PE,SRT PE,CRT PE
Test Used,Kruskal-Wallis Test (Non-Normal Distribution),Kruskal-Wallis Test (Non-Normal Distribution),Kruskal-Wallis Test (Non-Normal Distribution),Kruskal-Wallis Test (Non-Normal Distribution),Kruskal-Wallis Test (Non-Normal Distribution),Kruskal-Wallis Test (Non-Normal Distribution),Kruskal-Wallis Test (Non-Normal Distribution)
p-value,0.0,0.0,0.000002,0.0,0.0,0.0,0.0
